In [ ]:
using Pkg; Pkg.activate("..")

In [ ]:
using CreditRisk
using Distributions
using PyPlot
import Random

### Plotting inner level objective function

In [ ]:
n = 2500
c = 4
s = 5
l = 0.2
param = Parameter(n,c,s,l)
let (N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

    Ψ = init_Ψ()

    Zdist = MvNormal(S, 1)
    Z = zeros(S)
    pnc = zeros(N, C)
    phi0 = zeros(N, C+1)
    phi  = @view phi0[:,2:end]
    twist = zeros(N, C)
    mgf = zeros(N)
    qnc = zeros(N, C)


    Random.rand!(Zdist, Z)
    @. phi = normcdf((H - $(β*Z)) / denom)
    diff!(pnc, phi0; dims=2)

    let objective(θ) = begin
        θ = θ[1]
        Ψ(θ, pnc, weights) - θ*l
    end

        xs = -1000:1000
        ys = [objective([x]) for x in xs]
        display(plot(xs, ys))
    end
end

### Plotting outer level objective function

In [ ]:

s = 2         
range = 5     # [-range, range]
step = 0.1
tails = [0.2]

for tail in tails
    param = Parameter(n,c,s,tail)
    let (N, C, S, loss, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

    phi0 = zeros(N, C+1)
    phi  = @view phi0[:,2:end]
    pnc = zeros(N, C)

    Ψ = init_Ψ()
    innerlevel = InnerLevelTwisting(N, C)

    let objective(z) = begin
        @. phi = normcdf((H - $(β*z)) / denom)
        diff!(pnc, phi0; dims=2)
        twist!(innerlevel, pnc, weights, loss)
        θ = get_result(innerlevel)
        θ*l - Ψ(θ, pnc, weights) + 0.5z'z
    end

        xs = -range:step:range
        ys = -range:step:range
        zs = [objective([x, y]) for x in xs, y in ys]

        display(surf(xs, ys, zs))
        savefig("surf_$loss.pdf", format="pdf")
        end
    end
end